# Dataset Linting Tutorial


## Problem Statement

Exploratory data analysis (EDA) can be overwhelming. There are so many things to check.
Duplicates in your dataset, bad/corrupted images in the set, blurred or bright/dark images, the list goes on.

DataEval created a Linting class to assist you with your EDA so you can start training your models on high quality data.


### _When to use_

The Linting class should be used during the initial EDA process or if you are trying to verify that you have the right data in your dataset.


### _What you will need_

1. A dataset to analyze
2. A Python environment with the following packages installed:
   - `dataeval` or `dataeval[all]`


## _Getting Started_

Let's import the required libraries needed to set up a minimal working example


In [1]:
try:
    import google.colab  # noqa: F401

    # specify the version of DataEval (==X.XX.X) for versions other than the latest
    %pip install -q dataeval
except Exception:
    pass

In [2]:
import torch
import torchvision.transforms.v2 as v2

from dataeval.detectors.linters import Outliers
from dataeval.utils.data.datasets import CIFAR10

## Loading in the data

We are going to start by loading in torchvision's CIFAR-10 dataset.

The CIFAR-10 dataset contains 60,000 images - 50,000 in the train set and 10,000 in the test set.
For the purposes of this demonstration, we are just going to use the test set.


In [ ]:
# Load in the cifar-10 dataset from torchvision
to_tensor = v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])
testing_dataset = CIFAR10("./data", train=False, download=True, transform=to_tensor)


## Linting the Dataset

Now we can begin finding those images which are significantly different from the rest of the data.


In [ ]:
# Initialize the Duplicates class
outliers = Outliers(outlier_method="zscore", outlier_threshold=3)

# Evaluate the data
results = outliers.evaluate(datum[0] for datum in testing_dataset)

The results are a dictionary with the keys being the image that has an issue in one of the listed properties below:

- Brightness
- Blurriness
- Missing
- Zero
- Width
- Height
- Size
- Aspect Ratio
- Channels
- Depth


In [ ]:
print(f"Total number of images with an issue: {len(results.issues)}")

In [ ]:
# Show a count of issues by type
issue_count_by_type = {}
for index, issue in results.issues.items():
    for k, v in issue.items():
        issue_count_by_type[k] = issue_count_by_type.setdefault(k, 0) + 1
for issue, count in issue_count_by_type.items():
    print(f"{issue:>10}: {count:<5}")

In [7]:
### TEST ASSERTION CELL ###
assert len(results.issues) == 501
assert {
    "var",
    "mean",
    "skew",
    "kurtosis",
    "entropy",
    "brightness",
    "contrast",
    "zeros",
    "sharpness",
    "std",
    "darkness",
} == set(issue_count_by_type)